# Imports

In [13]:
# %%time
from os import listdir
from os.path import isfile, join
# from pprint import pprint 
import pandas as pd
# import json

CPU times: user 594 ms, sys: 139 ms, total: 733 ms
Wall time: 2.43 s


# Read all files in `data` directory

In [24]:
# read all data files

mypath = 'data'
data_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(data_files)

[&#39;access.log.11&#39;, &#39;access.log.4&#39;, &#39;access.log.9&#39;, &#39;access.log.6&#39;, &#39;access.log.14&#39;, &#39;access.log.7&#39;, &#39;access.log.13&#39;, &#39;access.log&#39;, &#39;access.log.10&#39;, &#39;access.log.1&#39;, &#39;access.log.3&#39;, &#39;access.log.2&#39;, &#39;access.log.12&#39;, &#39;access.log.5&#39;, &#39;access.log.8&#39;]


## Add all files data into one Pandas dataframe

In [26]:
li = []
for file in data_files:
    tmp = pd.read_csv(f'{mypath}/{file}', sep='\s+', header=None)
    li.append(tmp)
df = pd.concat(li, axis=0, ignore_index=True)

# Explore and prepare data

In [27]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,68.183.188.21,-,-,[15/Sep/2020:06:32:49,+0200],5\xF8kk\x1A!\x91UB\x94Q\xA9:\xC2wl,400,182,-,-
1,68.183.188.21,-,-,[15/Sep/2020:06:32:50,+0200],\x08\x86w<W\xA00\x5C,400,182,-,-
2,37.151.167.200,-,-,[15/Sep/2020:06:48:08,+0200],GET / HTTP/1.1,200,612,-,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...
3,45.141.84.72,-,-,[15/Sep/2020:06:49:30,+0200],\x03\x00\x00/*\xE0\x00\x00\x00\x00\x00Cookie: ...,400,182,-,-
4,172.104.108.109,-,-,[15/Sep/2020:07:10:35,+0200],GET / HTTP/1.1,200,396,-,Mozilla/5.0


In [28]:
df.shape

(2900, 10)

### remove columns holding useless data '-'

In [29]:
df[1].describe()

count     2900
unique       1
top          -
freq      2900
Name: 1, dtype: object

In [31]:
rm_colls =[1,2,4,8]
df.drop(df.columns[rm_colls], axis=1, inplace=True)

IndexError: index 8 is out of bounds for axis 0 with size 6

Give names to columns

In [38]:
columns = {0:'IP',3:'date',5:'request',6:'code',7:'body_bytes',9:'browser'}
df.rename(columns=columns,inplace=True)

remove '[' from date_time column

In [33]:
df['date'] = df['date'].str[1:]

In [39]:
df.head()

,IP,date,request,code,unkown,browser
0,68.183.188.21,15/Sep/2020:06:32:49,5\xF8kk\x1A!\x91UB\x94Q\xA9:\xC2wl,400,182,-
1,68.183.188.21,15/Sep/2020:06:32:50,\x08\x86w<W\xA00\x5C,400,182,-
2,37.151.167.200,15/Sep/2020:06:48:08,GET / HTTP/1.1,200,612,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...
3,45.141.84.72,15/Sep/2020:06:49:30,\x03\x00\x00/*\xE0\x00\x00\x00\x00\x00Cookie: ...,400,182,-
4,172.104.108.109,15/Sep/2020:07:10:35,GET / HTTP/1.1,200,396,Mozilla/5.0


# (2) Data labeling

The source of access logs server is where the Python Flask application runs.

## (2.1) Function used to filter malicious requests using keywords 

In [83]:
def find_bad(tmp_df, phrase):
    'Find rows by keyword phrase using regex'
    new_df = tmp_df[tmp_df.requests.str.contains(phrase, regex=True, na=False)]
    return new_df

In [97]:
def build_phrase(list):
    tmp = ''
    for word in list:
        if tmp == '': tmp = f'{word}'
        else: tmp = f'{tmp}|{word}'
    return tmp

### (2.1.1) all bad requests

In [103]:
bad_requests_phrase = build_phrase([
    'php', 'wp', 'cgi', 'shell',
    'ls+', 'cd\+', 'rm\+', '\-rf', 'wget\+', 'cmod\+', 'chown\+',
    '\?index=','example.com', 'ZINFO', 'fig'
])
bad_requests_phrase

&#39;php|wp|cgi|shell|ls+|cd\\+|rm\\+|\\-rf|wget\\+|cmod\\+|chown\\+|\\?index=|example.com|ZINFO|fig&#39;

In [104]:
bad_requests = df[df.request.str.contains(bad_requests_phrase, regex=True, na=False)]

How many bad requests found in overall dataset?

In [105]:
bad_requests.shape

(1110, 6)

TODO - do we have overlaping results because of few keywords in one request? filter out dublicated rows

## (2.2) Tests on filtering bad requests

### (2.2.1) Find malitious requests accessing *.php files

In [49]:
# php_requests = df.loc[df.request.str.contains('^.php')]
# df.request.str.contains('^.php^')
bad_php = df[df.request.str.contains('php', regex=True, na=False)]

In [50]:
bad_php.shape

(1033, 6)

### (2.2.2) Find malitiuos requests trying to execute shell commands

In [79]:
bad_shell = df[df.request.str.contains('shell|cd\+|rm\+|\-rf|wget\+|chmod\+|chown\+|ls\+', regex=True, na=False)]

In [80]:
bad_shell.shape

(34, 6)

### (2.2.3) Find malitious requests of CGI

In [63]:
bad_cgi = df[df.request.str.contains('cgi', regex=True, na=False)]

In [106]:
bad_cgi.shape

(34, 6)

### (2.2.4) Other malitious

In [88]:
bad_other = df[df.request.str.contains('\?index=|example\.com|ZINFO|fig', regex=True, na=False)]

In [89]:
bad_other.shape

(69, 6)

## (2.3) Good requests

In [113]:
good_requests = bad_requests.merge(df, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']

In [114]:
good_requests.shape

(1790, 7)

In [115]:
good_requests.head()

,IP,date,request,code,unkown,browser,_merge
1118,68.183.188.21,15/Sep/2020:06:32:49,5\xF8kk\x1A!\x91UB\x94Q\xA9:\xC2wl,400,182,-,right_only
1119,68.183.188.21,15/Sep/2020:06:32:50,\x08\x86w<W\xA00\x5C,400,182,-,right_only
1120,37.151.167.200,15/Sep/2020:06:48:08,GET / HTTP/1.1,200,612,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,right_only
1121,45.141.84.72,15/Sep/2020:06:49:30,\x03\x00\x00/*\xE0\x00\x00\x00\x00\x00Cookie: ...,400,182,-,right_only
1122,172.104.108.109,15/Sep/2020:07:10:35,GET / HTTP/1.1,200,396,Mozilla/5.0,right_only


# (3) Model

## (3.1) Splitting the date (training / validation)